<a href="https://colab.research.google.com/github/Djadih/DataScience_Project/blob/main/midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [202]:
from google.colab import files
uploaded = files.upload()

Saving mz_331_1.mid to mz_331_1.mid


In [72]:
import string
import numpy as np
from music21 import converter, instrument, note, chord, stream, tempo, meter

In [148]:
midi = converter.parse('beethoven_opus10_1.mid')

In [ ]:
max = 530
j = 0
for i in midi.parts[0].recurse():
  print(i, i.offset)
  j+=1
  if j ==max:
    break

In [ ]:
 for i in midi.flat.notes:
   print(i)

In [212]:
def get_notes(filename):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    temp = 0
    temp_string = ''
    temp_found = False
    st = ''
    midi = converter.parse(filename)
    print("Parsing %s" % filename)
    notes_to_parse = None

    try: # file has instrument parts
      ins = instrument.partitionByInstrument(midi)
      notes_to_parse = ins.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, tempo.MetronomeMark) and not temp_found:
        temp = element.getQuarterBPM()
        temp_string = element.text
        temp_found = True
      elif isinstance(element, meter.TimeSignature):
        st = element.ratioString

      elif isinstance(element, note.Note):
        notes.append((str(element.pitch), element.offset))
      elif isinstance(element, chord.Chord):
        notes.append(('.'.join(str(n) for n in element.normalOrder), element.offset))

    return notes, temp, temp_string, st

def get_dict(filename):
    notes, temp, temp_string, st = get_notes(filename)
    music = {}
    music['tempo_num'] = temp
    music['tempo_string'] = temp_string
    music['time_signature'] = st
    return music, notes

In [213]:
song, notes = get_dict('mz_331_1.mid')

Parsing mz_331_1.mid


In [214]:
song['tempo_string']

In [220]:
for i in notes[:20]:
  print(i)

('C#5', 0.0)
('A3', 0.0)
('E4', 0.0)
('D5', 0.75)
('B3', 0.75)
('C#5', 1.0)
('4.9', 1.0)
('E5', 1.5)
('1.4', 1.5)
('E5', 2.5)
('1.4', 2.5)
('B4', 3.0)
('G#3', 3.0)
('E4', 3.0)
('C#5', 3.75)
('A3', 3.75)
('B4', 4.0)
('4.8', 4.0)
('D5', 4.5)
('11.4', 4.5)


In [218]:
def create_midi(prediction_dict, notes):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    output_notes = []
    temp_num = prediction_dict['tempo_num']
    temp_string = prediction_dict['tempo_string']
    st = prediction_dict['time_signature']

    mm1 = tempo.MetronomeMark(temp_string, temp_num)
    ts = meter.TimeSignature(st)
    output_notes.append(mm1)
    output_notes.append(ts)

    # create note and chord objects based on the values generated by the model
    for pattern, offset in notes:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that not  es do not stack

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output_tempo.mid')

In [219]:
create_midi(song, notes)